# 📄 Génération de Rapports HTML Professionnels

## Introduction

Dans ce dernier notebook, nous allons combiner toutes les analyses précédentes pour générer un rapport HTML professionnel, similaire à celui produit par votre système Heston.v2.

### 🎯 Objectifs
1. Créer des visualisations de qualité publication
2. Générer un rapport HTML interactif et esthétique
3. Comprendre le workflow complet de bout en bout

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import pandas as pd
import base64
from io import BytesIO
from datetime import datetime, timedelta

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")
%matplotlib inline

In [ ]:
# Notre classe Heston
class HestonModel:
    def __init__(self, S0, V0, mu, kappa, theta, sigma_v, rho):
        self.S0 = S0
        self.V0 = V0
        self.mu = mu
        self.kappa = kappa
        self.theta = theta
        self.sigma_v = sigma_v
        self.rho = rho
        self.params = {
            'S0': S0, 'V0': V0, 'mu': mu,
            'kappa': kappa, 'theta': theta,
            'sigma_v': sigma_v, 'rho': rho
        }
    
    def simulate(self, T=1.0, N=1000, n_paths=1, seed=None):
        if seed is not None:
            np.random.seed(seed)
        
        dt = T / N
        sqrt_dt = np.sqrt(dt)
        t = np.linspace(0, T, N+1)
        
        S = np.zeros((N+1, n_paths))
        V = np.zeros((N+1, n_paths))
        S[0] = self.S0
        V[0] = self.V0
        
        rho_comp = np.sqrt(1 - self.rho**2)
        
        for i in range(N):
            Z1 = np.random.standard_normal(n_paths)
            Z2 = np.random.standard_normal(n_paths)
            
            dW_S = sqrt_dt * Z1
            dW_v = sqrt_dt * (self.rho * Z1 + rho_comp * Z2)
            
            V_current = np.maximum(V[i], 0)
            sqrt_V = np.sqrt(V_current)
            
            V[i+1] = V[i] + self.kappa * (self.theta - V_current) * dt \
                     + self.sigma_v * sqrt_V * dW_v
            V[i+1] = np.maximum(V[i+1], 0)
            
            S[i+1] = S[i] * np.exp((self.mu - 0.5*V_current) * dt + sqrt_V * dW_S)
        
        return t, S, V

## 1. Fonction de génération de graphiques pour HTML

In [ ]:
def fig_to_base64(fig):
    """Convertit une figure matplotlib en string base64 pour HTML"""
    buffer = BytesIO()
    fig.savefig(buffer, format='png', dpi=150, bbox_inches='tight')
    buffer.seek(0)
    img_base64 = base64.b64encode(buffer.read()).decode('utf-8')
    plt.close(fig)
    return img_base64

def create_histogram_plot(prix_finaux, S0, percentiles, target_price=None):
    """Crée l'histogramme principal de distribution"""
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Histogramme
    counts, bins, patches = ax.hist(prix_finaux, bins=80, alpha=0.75, 
                                      edgecolor='black', color='steelblue',
                                      density=False)
    
    # Lignes de percentiles
    colors_perc = {'p5': 'red', 'p25': 'orange', 'p50': 'green', 
                   'p75': 'blue', 'p95': 'purple'}
    for key, color in colors_perc.items():
        ax.axvline(percentiles[key], color=color, linestyle='--', 
                  linewidth=2, alpha=0.8, label=f'{key.upper()}: {percentiles[key]:,.0f}')
    
    # Moyenne
    mean_price = np.mean(prix_finaux)
    ax.axvline(mean_price, color='black', linestyle='--', linewidth=2.5,
              label=f'Moyenne: {mean_price:,.0f}')
    
    # Target price si fourni
    if target_price:
        ax.axvline(target_price, color='darkred', linestyle=':', linewidth=3,
                  label=f'Cible: {target_price:,.0f}')
    
    ax.set_title('Distribution des Prix Finaux - Simulation Monte Carlo Heston', 
                fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Prix (USD)', fontsize=13)
    ax.set_ylabel('Fréquence', fontsize=13)
    ax.legend(loc='upper right', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    return fig

def create_paths_plot(t, S_paths, percentiles_paths, S0):
    """Crée le graphique des trajectoires Monte Carlo"""
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Afficher quelques trajectoires
    n_display = min(200, S_paths.shape[1])
    for i in range(n_display):
        ax.plot(t, S_paths[:, i], alpha=0.05, linewidth=0.5, color='gray')
    
    # Percentiles
    ax.plot(t, percentiles_paths['p50'], 'g-', linewidth=3, label='P50 (Médiane)')
    ax.plot(t, percentiles_paths['p10'], 'r--', linewidth=2, label='P10')
    ax.plot(t, percentiles_paths['p90'], 'b--', linewidth=2, label='P90')
    ax.fill_between(t, percentiles_paths['p10'], percentiles_paths['p90'],
                    alpha=0.2, color='lightblue', label='Intervalle 80%')
    
    # Prix initial
    ax.axhline(y=S0, color='black', linestyle=':', linewidth=2, label='Prix initial')
    
    ax.set_title('Trajectoires de Prix - Simulation Heston', 
                fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Temps (jours)', fontsize=13)
    ax.set_ylabel('Prix (USD)', fontsize=13)
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    return fig

## 2. Simulation complète du scénario

In [ ]:
# Paramètres du scénario (Bitcoin à 30 jours)
print("🚀 Configuration du scénario...\n")

# Marché
symbol = "BTCUSD"
S0 = 95000
forecast_days = 30
target_price = 100000  # Seuil d'intérêt

# Paramètres Heston (calibrés)
V0 = 0.16  # variance initiale (40% vol)
mu = 0.15  # drift annuel
kappa = 2.5
theta = 0.16
sigma_v = 0.5
rho = -0.7

# Simulation
n_simulations = 20000
n_steps = forecast_days

print(f"📊 Paramètres:")
print(f"   Symbol: {symbol}")
print(f"   Prix initial: ${S0:,}")
print(f"   Horizon: {forecast_days} jours")
print(f"   Simulations: {n_simulations:,}")
print(f"   Target: ${target_price:,}")
print()

# Créer modèle et simuler
model = HestonModel(S0, V0, mu, kappa, theta, sigma_v, rho)
t, S_paths, V_paths = model.simulate(
    T=forecast_days/365, 
    N=n_steps, 
    n_paths=n_simulations,
    seed=42
)

# Convertir temps en jours
t_days = t * 365

prix_finaux = S_paths[-1, :]
print("✅ Simulation terminée!")

## 3. Calcul des statistiques complètes

In [ ]:
# Statistiques
stats_dict = {
    'mean': np.mean(prix_finaux),
    'median': np.median(prix_finaux),
    'std': np.std(prix_finaux),
    'min': np.min(prix_finaux),
    'max': np.max(prix_finaux),
    'ci_95': (np.percentile(prix_finaux, 2.5), np.percentile(prix_finaux, 97.5))
}

percentiles = {
    'p5': np.percentile(prix_finaux, 5),
    'p10': np.percentile(prix_finaux, 10),
    'p25': np.percentile(prix_finaux, 25),
    'p50': np.percentile(prix_finaux, 50),
    'p75': np.percentile(prix_finaux, 75),
    'p90': np.percentile(prix_finaux, 90),
    'p95': np.percentile(prix_finaux, 95)
}

# Percentiles temporels
percentiles_paths = {
    'p10': np.percentile(S_paths, 10, axis=1),
    'p50': np.percentile(S_paths, 50, axis=1),
    'p90': np.percentile(S_paths, 90, axis=1)
}

# Probabilités
prob_above_target = (prix_finaux > target_price).mean()
p = prob_above_target
n = len(prix_finaux)
se = np.sqrt(p * (1-p) / n)
prob_ci = (max(0, p - 1.96*se), min(1, p + 1.96*se))

# Binary markets (différents seuils)
binary_markets = {}
for threshold in [80000, 90000, 100000, 110000, 120000]:
    prob = (prix_finaux > threshold).mean()
    se_t = np.sqrt(prob * (1-prob) / n)
    ci = (max(0, prob - 1.96*se_t), min(1, prob + 1.96*se_t))
    binary_markets[threshold] = {'probability': prob, 'ci': ci}

# Bucket markets (intervalles)
bucket_markets = {}
buckets = [(0, 85000), (85000, 95000), (95000, 105000), (105000, 115000), (115000, 200000)]
for low, high in buckets:
    prob = ((prix_finaux >= low) & (prix_finaux < high)).mean()
    se_b = np.sqrt(prob * (1-prob) / n)
    ci = (max(0, prob - 1.96*se_b), min(1, prob + 1.96*se_b))
    bucket_markets[(low, high)] = {'probability': prob, 'ci': ci}

print("✅ Statistiques calculées")

## 4. Génération des graphiques

In [ ]:
print("🎨 Génération des graphiques...\n")

# Histogramme
fig_hist = create_histogram_plot(prix_finaux, S0, percentiles, target_price)
hist_b64 = fig_to_base64(fig_hist)
print("  ✓ Histogramme créé")

# Trajectoires
fig_paths = create_paths_plot(t_days, S_paths, percentiles_paths, S0)
paths_b64 = fig_to_base64(fig_paths)
print("  ✓ Graphique des trajectoires créé")

print("\n✅ Tous les graphiques générés")

## 5. Génération du rapport HTML

In [ ]:
def generate_html_report(stats, percentiles, binary_markets, bucket_markets, 
                         hist_b64, paths_b64, model_params, config):
    """
    Génère un rapport HTML complet style Heston.v2
    """
    # Rows pour percentiles
    perc_rows = "\n".join([
        f"<tr><td>{k.upper()}</td><td>{v:,.2f}</td></tr>"
        for k, v in percentiles.items()
    ])
    
    # Rows pour binary markets
    binary_rows = "\n".join([
        f"""<tr>
            <td>{k:,}</td>
            <td>{v['probability']*100:.2f}%<br/>
                <small>[{v['ci'][0]*100:.2f}%, {v['ci'][1]*100:.2f}%]</small>
            </td>
        </tr>"""
        for k, v in binary_markets.items()
    ])
    
    # Rows pour buckets
    bucket_rows = "\n".join([
        f"""<tr>
            <td>[{k[0]:,}, {k[1]:,})</td>
            <td>{v['probability']*100:.2f}%<br/>
                <small>[{v['ci'][0]*100:.2f}%, {v['ci'][1]*100:.2f}%]</small>
            </td>
        </tr>"""
        for k, v in bucket_markets.items()
    ])
    
    # Paramètres modèle
    param_rows = "\n".join([
        f"""<tr>
            <td>{name}</td>
            <td>{value:.4f}</td>
            <td>{desc}</td>
        </tr>"""
        for name, value, desc in [
            ('S₀', model_params['S0'], 'Prix initial'),
            ('V₀', model_params['V0'], 'Variance initiale'),
            ('μ', model_params['mu'], 'Drift (rendement moyen)'),
            ('κ', model_params['kappa'], 'Vitesse de retour à la moyenne'),
            ('θ', model_params['theta'], 'Variance de long terme'),
            ('σᵥ', model_params['sigma_v'], 'Volatilité de la volatilité'),
            ('ρ', model_params['rho'], 'Corrélation (effet de levier)')
        ]
    ])
    
    html = f"""
<!DOCTYPE html>
<html lang="fr">
<head>
    <meta charset="UTF-8">
    <title>Rapport Heston - {config['symbol']} - {datetime.now():%Y-%m-%d}</title>
    <style>
        body {{ 
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; 
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: #e2e8f0; 
            margin: 0; 
            padding: 40px; 
        }}
        .container {{ 
            max-width: 1200px; 
            margin: auto; 
            background: #1e293b; 
            border-radius: 20px; 
            overflow: hidden; 
            box-shadow: 0 20px 60px rgba(0,0,0,0.5); 
        }}
        .header {{ 
            background: linear-gradient(135deg, #2563eb, #7c3aed); 
            padding: 40px; 
            text-align: center;
        }}
        .header h1 {{ margin: 0; font-size: 2.5em; }}
        .header p {{ margin: 10px 0 0 0; font-size: 1.1em; opacity: 0.9; }}
        .grid {{ 
            display: grid; 
            grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); 
            gap: 20px; 
            padding: 30px; 
        }}
        .stat-card {{ 
            background: #0f172a; 
            padding: 25px; 
            border-radius: 15px; 
            border: 1px solid rgba(148,163,184,0.2); 
            transition: transform 0.3s, box-shadow 0.3s;
        }}
        .stat-card:hover {{
            transform: translateY(-5px);
            box-shadow: 0 10px 30px rgba(59,130,246,0.3);
        }}
        .stat-label {{ 
            font-size: 0.9em; 
            color: #94a3b8; 
            margin-bottom: 8px; 
            text-transform: uppercase; 
            letter-spacing: 1px;
        }}
        .stat-value {{ 
            font-size: 2em; 
            font-weight: 700; 
            color: #4ade80;
        }}
        .stat-sub {{ 
            font-size: 0.85em; 
            color: #94a3b8; 
            margin-top: 5px; 
        }}
        .section {{ padding: 30px; }}
        .section h2 {{ 
            margin-top: 0; 
            font-size: 1.8em;
            border-bottom: 3px solid #3b82f6;
            padding-bottom: 10px;
        }}
        table {{ 
            width: 100%; 
            border-collapse: collapse; 
            margin-top: 20px;
            border-radius: 10px;
            overflow: hidden;
        }}
        th {{ 
            background: rgba(59,130,246,0.2); 
            padding: 15px; 
            text-align: left;
            text-transform: uppercase;
            letter-spacing: 0.5px;
            font-size: 0.9em;
        }}
        td {{ 
            padding: 12px 15px; 
            border-bottom: 1px solid rgba(148,163,184,0.1); 
        }}
        tbody tr:nth-child(odd) {{ background: rgba(15,23,42,0.4); }}
        tbody tr:hover {{ background: rgba(96,165,250,0.15); }}
        .image {{ text-align: center; padding: 20px; }}
        .image img {{ 
            width: 100%; 
            max-width: 1000px;
            border-radius: 15px; 
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        }}
        .badge {{ 
            display: inline-block; 
            padding: 8px 16px; 
            border-radius: 20px; 
            background: #475569; 
            font-size: 0.85em;
            font-weight: 600;
        }}
        .highlight {{ color: #fbbf24; }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>📊 Rapport de Prévision Heston</h1>
            <p>{config['symbol']} · Horizon {config['forecast_days']} jours · {n_simulations:,} simulations</p>
            <p><span class="badge">{datetime.now():%Y-%m-%d %H:%M}</span></p>
        </div>
        
        <div class="grid">
            <div class="stat-card">
                <div class="stat-label">Prix Initial</div>
                <div class="stat-value">${config['S0']:,.0f}</div>
            </div>
            <div class="stat-card">
                <div class="stat-label">Prix Moyen (T={config['forecast_days']}j)</div>
                <div class="stat-value">${stats['mean']:,.0f}</div>
            </div>
            <div class="stat-card">
                <div class="stat-label">Médiane</div>
                <div class="stat-value">${stats['median']:,.0f}</div>
            </div>
            <div class="stat-card">
                <div class="stat-label">P(Prix > ${config['target_price']:,})</div>
                <div class="stat-value highlight">{prob_above_target*100:.2f}%</div>
                <div class="stat-sub">IC 95%: [{prob_ci[0]*100:.2f}%, {prob_ci[1]*100:.2f}%]</div>
            </div>
        </div>
        
        <div class="section image">
            <h2>Distribution des Prix Finaux</h2>
            <img src="data:image/png;base64,{hist_b64}" alt="Histogramme" />
            <p>Intervalle de confiance 95%: [${stats['ci_95'][0]:,.0f}, ${stats['ci_95'][1]:,.0f}]</p>
        </div>
        
        <div class="section image">
            <h2>Trajectoires Monte Carlo</h2>
            <img src="data:image/png;base64,{paths_b64}" alt="Trajectoires" />
        </div>
        
        <div class="section">
            <h2>Paramètres du Modèle de Heston</h2>
            <table>
                <thead><tr><th>Paramètre</th><th>Valeur</th><th>Description</th></tr></thead>
                <tbody>{param_rows}</tbody>
            </table>
        </div>
        
        <div class="section">
            <h2>Percentiles de Prix</h2>
            <table>
                <thead><tr><th>Percentile</th><th>Prix (USD)</th></tr></thead>
                <tbody>{perc_rows}</tbody>
            </table>
        </div>
        
        <div class="section">
            <h2>Probabilités Binaires P(Prix > K)</h2>
            <table>
                <thead><tr><th>Seuil K (USD)</th><th>Probabilité (IC 95%)</th></tr></thead>
                <tbody>{binary_rows}</tbody>
            </table>
        </div>
        
        <div class="section">
            <h2>Distribution par Intervalles</h2>
            <table>
                <thead><tr><th>Intervalle</th><th>Probabilité (IC 95%)</th></tr></thead>
                <tbody>{bucket_rows}</tbody>
            </table>
        </div>
        
        <div class="section" style="text-align: center; padding: 40px;">
            <p style="font-size: 0.9em; color: #94a3b8;">
                Rapport généré avec le modèle de Heston (volatilité stochastique)<br/>
                Notebook pédagogique - Heston Learning Lab 🚀
            </p>
        </div>
    </div>
</body>
</html>
    """
    return html

# Générer le rapport
config = {
    'symbol': symbol,
    'S0': S0,
    'forecast_days': forecast_days,
    'target_price': target_price
}

html_report = generate_html_report(
    stats_dict, percentiles, binary_markets, bucket_markets,
    hist_b64, paths_b64, model.params, config
)

# Sauvegarder
output_path = 'heston_report.html'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(html_report)

print(f"\n✅ Rapport HTML généré: {output_path}")
print(f"   Ouvrez le fichier dans votre navigateur pour voir le résultat!")

## 🎉 Félicitations !

Vous avez complété le laboratoire d'apprentissage du modèle de Heston !

### 🎓 Ce que vous avez appris :

1. **Notebook 01** : Concepts fondamentaux (variables aléatoires, distributions, simulations)
2. **Notebook 02** : Mouvement Brownien (browniens corrélés, propriétés mathématiques)
3. **Notebook 03** : Modèle de Heston complet (équations, paramètres, implémentation)
4. **Notebook 04** : Simulations Monte Carlo (statistiques, probabilités, percentiles)
5. **Notebook 05** : Génération de rapports (visualisations, HTML)

### 🔑 Compétences acquises :

- ✅ Comprendre la théorie derrière les modèles de volatilité stochastique
- ✅ Implémenter le modèle de Heston en Python
- ✅ Générer des simulations Monte Carlo
- ✅ Analyser et interpréter les résultats
- ✅ Créer des rapports professionnels

### 🚀 Pour aller plus loin :

1. **Calibration** : Estimer les paramètres à partir de données réelles
2. **Pricing d'options** : Utiliser Heston pour valoriser des options
3. **Gestion de risque** : VaR, Expected Shortfall
4. **Trading algorithmique** : Signaux basés sur la volatilité implicite

### 📚 Ressources :

- Article original : Heston, S. L. (1993). "A Closed-Form Solution for Options with Stochastic Volatility"
- Livre : "The Volatility Surface" par Jim Gatheral
- Code source : Votre projet Heston.v2 !

---

**Merci d'avoir suivi ce laboratoire ! N'hésitez pas à expérimenter avec différents paramètres et scénarios.** 🎯